In [7]:
import pandas as pd
import numpy as np
import os

input_csv = "/home/franciscojavier/Escritorio/TFG/scDEAL/data/ALL_expression.csv"  
top_percentage = 0.2  # 20%
output_dir = "/home/franciscojavier/Escritorio/TFG/scDEAL/save/adata/"

filename_base = os.path.splitext(os.path.basename(input_csv))[0]
output_txt = os.path.join(output_dir, f"selected_genes_{filename_base}.txt")
output_csv = os.path.join(output_dir, f"expression_{filename_base}_top{int(top_percentage*100)}.csv")

print(f"Cargando matriz desde: {input_csv}")
df = pd.read_csv(input_csv, index_col=0)
variances = df.var(axis=0)
top_k = int(len(variances) * top_percentage)
top_genes = variances.sort_values(ascending=False).head(top_k).index.tolist()


os.makedirs(output_dir, exist_ok=True)
with open(output_txt, "w") as f:
    for gene in top_genes:
        f.write(f"{gene}\n")
print(f"Archivo de texto guardado: {output_txt}")

df_filtered = df[top_genes]
df_filtered.to_csv(output_csv)
print(f"CSV con expresión filtrada guardado: {output_csv}")


Cargando matriz desde: /home/franciscojavier/Escritorio/TFG/scDEAL/data/ALL_expression.csv
Archivo de texto guardado: /home/franciscojavier/Escritorio/TFG/scDEAL/save/adata/selected_genes_ALL_expression.txt
CSV con expresión filtrada guardado: /home/franciscojavier/Escritorio/TFG/scDEAL/save/adata/expression_ALL_expression_top20.csv


In [1]:
# Paso 1: Leer el archivo de genes seleccionados por bulk
bulk_genes_path = "/home/franciscojavier/Escritorio/TFG/scDEAL/save/adata/selected_genes_GSE110894.txt"

with open(bulk_genes_path, "r") as f:
    bulk_genes = [line.strip() for line in f]

print(f"Se han leído {len(bulk_genes)} genes del archivo.")
print("Ejemplo de genes:", bulk_genes[:10])


Se han leído 3192 genes del archivo.
Ejemplo de genes: ['MFSD6', 'GNA11', 'DUSP11', 'TCEAL5', 'IFNGR1', 'C2ORF42', 'RRP1B', 'ANGPT1', 'IGFL3', 'RAET1G']


In [5]:
pip install pybiomart


Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

# Leer los datos de single-cell (genes en las filas)
sc_path = "/home/franciscojavier/Escritorio/TFG/scDEAL/data/GSE110894/GSE110894.csv"
sc_df = pd.read_csv(sc_path, index_col=0)

# Extraer los nombres de los genes del índice (filas)
sc_genes = sc_df.index.tolist()

print(f"Número total de genes en single-cell: {len(sc_genes)}")
print("Ejemplo de genes en single-cell:")
print(sc_genes[:10])



Número total de genes en single-cell: 25921
Ejemplo de genes en single-cell:
['ENSMUSG00000110331', 'ENSMUSG00000006281', 'ENSMUSG00000040726', 'ENSMUSG00000024829', 'ENSMUSG00000029007', 'ENSMUSG00000032026', 'ENSMUSG00000031808', 'ENSMUSG00000042549', 'ENSMUSG00000024007', 'ENSMUSG00000037151']


In [6]:
from pybiomart import Server
import pandas as pd

# Paso 1: conectarse al servidor Ensembl
server = Server(host='http://www.ensembl.org')

# Paso 2: usar el dataset humano
dataset = server.marts['ENSEMBL_MART_ENSEMBL'].datasets['hsapiens_gene_ensembl']

# Paso 3: cargar tus genes humanos desde archivo
with open( "/home/franciscojavier/Escritorio/TFG/scDEAL/save/adata/selected_genes_ALL_expression.txt") as f:
    bulk_genes = [line.strip() for line in f]

print(f"Se cargaron {len(bulk_genes)} genes humanos.")

# Paso 4: hacer la consulta y filtrar solo los genes que tenés
results = dataset.query(attributes=[
    'external_gene_name',                        # Nombre del gen humano
    'ensembl_gene_id',                           # ID humano
    'mmusculus_homolog_ensembl_gene',            # ID de ratón
    'mmusculus_homolog_associated_gene_name'     # nombre del gen de ratón
])

# Paso 5: filtrar por tus genes
mapped = results[results['Gene name'].isin(bulk_genes)]

# Paso 6: eliminar filas sin homólogo
mapped = mapped.dropna(subset=['Mouse gene stable ID'])

# Paso 7: guardar el archivo con resultados
# Guardar el archivo en el mismo directorio que el notebook
mapped[['Mouse gene stable ID']].drop_duplicates().to_csv("genes_mouse_from_bulk.txt", index=False, header=False)


print(f"✅ Mapeo completado. Se encontraron {mapped.shape[0]} genes con homólogo en ratón.")


Se cargaron 3192 genes humanos.
✅ Mapeo completado. Se encontraron 3232 genes con homólogo en ratón.


In [14]:
print("Genes humanos únicos en entrada:", len(set(bulk_genes)))
print("Filas únicas en mapeo ratón:", mapped[['Mouse gene stable ID']].drop_duplicates().shape[0])


Genes humanos únicos en entrada: 3192
Filas únicas en mapeo ratón: 3189


In [16]:
import pandas as pd

# Ruta al archivo de expresión single-cell
sc_path = "/home/franciscojavier/Escritorio/TFG/scDEAL/data/GSE110894/GSE110894.csv"

# Ruta al archivo de genes ENSMUSG
mouse_genes_path = "/home/franciscojavier/Escritorio/TFG/scDEAL/casestudy/genes_mouse_from_bulk.txt"

# Leer el archivo de expresión (genes en las filas)
sc_df = pd.read_csv(sc_path, index_col=0)

# Leer los genes seleccionados (ya mapeados desde bulk)
with open(mouse_genes_path) as f:
    genes_ratón = [line.strip() for line in f]

print(f"Genes mapeados a conservar: {len(genes_ratón)}")

# Filtrar el DataFrame por los genes
sc_df_filtrado = sc_df.loc[sc_df.index.intersection(genes_ratón)]

print(f"Matriz filtrada: {sc_df_filtrado.shape}")

# Guardar el nuevo archivo filtrado
sc_df_filtrado.to_csv("/home/franciscojavier/Escritorio/TFG/scDEAL/data/GSE110894/GSE110894_filtrado.csv")
print("✅ Archivo filtrado guardado como GSE110894_filtrado.csv")


Genes mapeados a conservar: 3189
Matriz filtrada: (2254, 1504)
✅ Archivo filtrado guardado como GSE110894_filtrado.csv


In [18]:
# Genes humanos seleccionados (en orden original del bulk)
with open("/home/franciscojavier/Escritorio/TFG/scDEAL/save/adata/selected_genes_GSE110894.txt") as f:
    bulk_genes = [line.strip() for line in f]


In [19]:
# Leer el archivo de mapeo humano → ratón (ENSMUSG)
mapped_df = pd.read_csv("genes_mouse_from_bulk.txt", header=None)
mapped_genes = mapped_df[0].tolist()


In [2]:
import sys
!{sys.executable} -m pip install mygene


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.6 MB/s eta 0:00:00


In [29]:
!pip install mygene


In [3]:
import pandas as pd
import mygene

# 1. Leer los genes humanos desde el archivo
with open("/home/franciscojavier/Escritorio/TFG/scDEAL/save/adata/selected_genes_GSE110894.txt") as f:
    human_genes = [line.strip() for line in f]

# 2. Conectar a mygene y hacer la consulta
mg = mygene.MyGeneInfo()

# 3. Consulta de homólogos en ratón
results = mg.querymany(human_genes, scopes="symbol", fields="homologene", species="human")

# 4. Extraer nombres de genes de ratón
mapped_genes = []
for r in results:
    try:
        mouse_gene = [h[1] for h in r['homologene']['genes'] if h[0] == 10090]  # 10090 = Mus musculus
        if mouse_gene:
            mapped_genes.append((r['query'], mouse_gene[0]))
    except:
        continue

# 5. Guardar en CSV
df = pd.DataFrame(mapped_genes, columns=["Human_Gene", "Mouse_Gene"])
df.to_csv("human_to_mouse_orthologs.csv", index=False)

print("✅ Archivo guardado como human_to_mouse_orthologs.csv")
df.head()


Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
1 input query terms found dup hits:	[('ATP6AP1L', 2)]
26 input query terms found no hit:	['DDX58', 'C9ORF24', 'C5ORF38', 'CLECL1', 'FAM189B', 'C10ORF99', 'TMEM211', 'SMIM6', 'KIAA2026', 'C1


✅ Archivo guardado como human_to_mouse_orthologs.csv


,Human_Gene,Mouse_Gene
0,MFSD6,98682
1,GNA11,14672
2,DUSP11,72102
3,TCEAL5,331532
4,IFNGR1,15979


In [10]:
import pandas as pd
import mygene

# Leer genes humanos
with open("/home/franciscojavier/Escritorio/TFG/scDEAL/save/adata/selected_genes_GSE110894.txt") as f:
    human_genes = [line.strip() for line in f]

# Consulta con mygene
mg = mygene.MyGeneInfo()
res = mg.querymany(human_genes, scopes='symbol', fields='homologene,mouse.ensembl.gene', species='human')

# Procesar resultados
mapped = []
for entry in res:
    human = entry.get('query', None)
    mouse_info = entry.get('mouse', {})

    if isinstance(mouse_info, dict):
        ensembl_id = mouse_info.get('ensembl', {}).get('gene', None)
        if isinstance(ensembl_id, str) and ensembl_id.startswith('ENSMUSG'):
            mapped.append((human, ensembl_id))

    elif isinstance(mouse_info, list):
        for m in mouse_info:
            ensembl_id = m.get('ensembl', {}).get('gene', None)
            if isinstance(ensembl_id, str) and ensembl_id.startswith('ENSMUSG'):
                mapped.append((human, ensembl_id))

# Mostrar resultados
print(f"✅ Genes mapeados encontrados: {len(mapped)}")

# Guardar en archivo plano
with open("genes_human_to_mouse_ENSMUSG.txt", "w") as f:
    for _, gene in mapped:
        f.write(gene + "\n")

# Guardar también en CSV
df = pd.DataFrame(mapped, columns=["human_gene", "mouse_ENSMUSG"])
df.to_csv("genes_mapped_human_to_mouse.csv", index=False)


Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
1 input query terms found dup hits:	[('ATP6AP1L', 2)]
26 input query terms found no hit:	['DDX58', 'C9ORF24', 'C5ORF38', 'CLECL1', 'FAM189B', 'C10ORF99', 'TMEM211', 'SMIM6', 'KIAA2026', 'C1


✅ Genes mapeados encontrados: 0
